In [10]:
import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import load_images
import data_set
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
FLAGS = dict()
FLAGS['width'] = 160
FLAGS['height'] = 90
FLAGS['batch_size'] = 16
FLAGS['kernel_1_out'] = 64
FLAGS['kernel_2_out'] = 32
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 1
FLAGS['train_report_step'] = 1
FLAGS['val_report_step'] = 5



In [7]:
X_train, y_train, X_train_id = load_images.load_train()

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 318.85 seconds


In [8]:
del X_train_id

In [40]:
data = data_set.DataSet(X_train, y_train)

In [43]:
del X_train
del y_train

In [ ]:
with tf.Graph().as_default() as g:
    with tf.name_scope('data'):
        X = tf.placeholder(tf.float32, [None, None, None, 3], name="X_placeholder")
        Y = tf.placeholder(tf.float32, [None, FLAGS['n_classes']], name="Y_placeholder")
        #dropout = tf.placeholder(tf.float32, name='dropout')
        
    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    with tf.variable_scope('conv1') as scope:
        scope.reuse_variables()
        kernel = tf.Variable(tf.truncated_normal(shape=[FLAGS['width'], 
                                                        FLAGS['height'], 3, FLAGS['kernel_1_out']], 
                                                        name='kernel'))
        biases = tf.Variable(tf.random_normal(shape=[FLAGS['kernel_1_out']]), name="bias")
        conv = tf.nn.conv2d(X, kernel, strides=[1, 1, 1, 1], padding='SAME')
        conv1 = tf.nn.relu(conv + biases, name=scope.name)
        kernel1_summ = tf.summary.scalar('kernel1_mean', tf.reduce_mean(kernel))
        conv1_summ = tf.summary.scalar('conv1_mean', tf.reduce_mean(conv1))
        # output is of dimension 16 x HEIGHT x WIDTH x KERNEL_1_OUT
   
    with tf.variable_scope('pool1') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME')
    
    with tf.variable_scope('conv2') as scope:
        kernel = tf.Variable(
            tf.truncated_normal(shape=[FLAGS['conv2_input_width'], 
                                       FLAGS['conv2_input_height'], 
                                       FLAGS['kernel_1_out'], 
                                       FLAGS['kernel_2_out']], name='kernel'))

        biases = tf.Variable(tf.random_normal(shape=[FLAGS['kernel_2_out']]), name="bias")
        conv = tf.nn.conv2d(pool1, kernel, strides=[1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.relu(conv + biases, name=scope.name)
        kernel2_summ = tf.summary.scalar('kernel2_mean', tf.reduce_mean(kernel))
        conv2_summ = tf.summary.scalar('conv2_mean', tf.reduce_mean(conv2))

    with tf.variable_scope('pool2') as scope:
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                                padding='SAME') 



    
    with tf.variable_scope('fc') as scope:

        # reshape pool2 to 2 dimensional
        pool2 = tf.reshape(pool2, [FLAGS['batch_size'], -1])
        
        #7680 is shape of pool2 second axis. 
        w = tf.Variable(tf.truncated_normal(shape=[7680, 1024], name='kernel'))
        b = tf.Variable(tf.random_normal(shape=[1024]), name="bias")
        
        fc = tf.nn.relu(tf.matmul(pool2, w) + b, name='relu')
        fcw_summ = tf.summary.scalar('fcw_mean', tf.reduce_mean(w))
        fc_summ = tf.summary.scalar('fc', tf.reduce_mean(fc))
        #Ouput is (BATCH_SIZE, 1024)
        #fc = tf.nn.dropout(fc, dropout, name='relu_dropout')
    
    with tf.variable_scope('softmax_linear') as scope:
        w = tf.Variable(tf.truncated_normal(shape=[1024, FLAGS['n_classes']],name='weights'))
        softw_summ = tf.summary.scalar('softw_mean', tf.reduce_mean(w))
        b = tf.Variable(tf.random_normal(shape=[FLAGS['n_classes']], name='biases'))
        logits = tf.matmul(fc, w) + b

    with tf.name_scope('loss'):
        entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
        loss = tf.reduce_mean(entropy, name='loss')

        entropy_clipped = tf.clip_by_value(entropy, 1e-15, 1 - 1e-15)
        log_loss = tf.convert_to_tensor(tf.reduce_mean(tf.log(entropy_clipped)))

        loss_summ = tf.summary.scalar('loss', loss)
        log_loss_summ = tf.summary.scalar('log_loss', log_loss)


    optimizer = tf.train.AdamOptimizer(FLAGS['learning_rate']).minimize(loss, 
                                            global_step=global_step)

    merged = tf.summary.merge_all()
    

    with tf.Session(graph=g) as sess:
        init = tf.global_variables_initializer()

        sess.run(init)

        #saver = tf.train.Saver()
        # to visualize using TensorBoard
        #writer = tf.summary.FileWriter('./my_graph/mnist', sess.graph)
        #ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist_new/checkpoint'))
        # if that checkpoint exists, restore from checkpoint
        #if ckpt and ckpt.model_checkpoint_path:
         #   saver.restore(sess, ckpt.model_checkpoint_path)

        initial_step = global_step.eval()

        start_time = time.time()

        n_batches = data.num_examples / FLAGS['batch_size']

        total_loss = 0.0
        train_writer = tf.summary.FileWriter('test-summary',
                                          sess.graph)
        val_writer = tf.summary.FileWriter('val-summary',
                                          sess.graph)
        train_writer.flush()
        val_writer.flush()
        X_val, y_val = data.get_validation_set()
        
        for index in range(initial_step, int(n_batches * FLAGS['n_epochs'])): # train the model n_epochs times
            
            X_batch, y_batch = data.next_batch(FLAGS['batch_size'])
            
            if (index + 1) % FLAGS['train_report_step'] == 0:
                _, loss_batch, l_loss, summary = sess.run([optimizer, loss, log_loss, merged], 
                                    feed_dict={X: X_batch, Y:y_batch}) 
                total_loss += loss_batch
                #print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / FLAGS['train_report_step']))
                print('log loss for batch: {}'.format(l_loss))
                train_writer.add_summary(summary, index)
            else:
                _, loss_batch = sess.run([optimizer, loss], 
                                    feed_dict={X: X_batch, Y:y_batch}) 
                total_loss += loss_batch
            
            if (index + 1) % FLAGS['val_report_step'] == 0:
                
                l_loss, summary  = sess.run([log_loss, merged], 
                                    feed_dict={X: X_val, Y:y_val}) 
                print('validation log loss for batch: {}'.format(l_loss))
                val_writer.add_summary(summary, index)

        #saver.save(sess, 'checkpoints/convnet_mnist_new/mnist-convnet', index)
        #print(sess.run([loss], feed_dict={X:X_train[:10, :,:,:], Y:y_mod[:10]}))
        #n_batches = int(mnist.train.num_examples / BATCH_SIZE)



log loss for batch: -2.1586735248565674
log loss for batch: 0.0
log loss for batch: -6.476020812988281
log loss for batch: -6.476020812988281
log loss for batch: -10.793367385864258
